In [9]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, HTML

# Define paths
data_paths = {
    "Agriculture": r"C:\Users\DELL\Downloads\Precipitation Data\Agriculture",
    "Winter": r"C:\Users\DELL\Downloads\Precipitation Data\winter",
    "Growing Season": r"C:\Users\DELL\Downloads\Precipitation Data\growing season",
}
excel_file_path = r"C:\Users\DELL\Downloads\actualdata_cleaned.xlsx"  # Path to your cleaned data Excel file

# Load cleaned data from the Excel sheets
cleaned_data = {
    "Agriculture": pd.read_excel(excel_file_path, sheet_name='agriculture'),
    "Winter": pd.read_excel(excel_file_path, sheet_name='winter'),
    "Growing Season": pd.read_excel(excel_file_path, sheet_name='growing season'),
}

# Create dropdown for dataset selection
dataset_dropdown = widgets.Dropdown(options=list(data_paths.keys()), description='Season:')
date_dropdown = widgets.Dropdown(description='Select Date:')
output = widgets.Output()

# Function to populate dates based on the selected dataset
def update_dates(change):
    dataset = dataset_dropdown.value
    if dataset in cleaned_data:
        dates = pd.to_datetime(cleaned_data[dataset]['Date']).dt.strftime('%Y-%m-%d').tolist()
        date_dropdown.options = dates
        date_dropdown.value = None  # Reset date selection to None to prevent automatic selection
    else:
        date_dropdown.options = []
        date_dropdown.value = None  # Reset date selection

# Function to read GeoTIFF data
def read_geotiff(file_path):
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the first band
        return data

# Function to show data based on selected date and dataset
def show_data(change):
    with output:
        output.clear_output()  # Clear previous output
        selected_dataset = dataset_dropdown.value
        selected_date = date_dropdown.value
        
        # Retrieve and display corresponding value from cleaned Excel data
        csv_value = cleaned_data[selected_dataset].loc[cleaned_data[selected_dataset]['Date'] == selected_date, 'Precipitation (mm)'].values
        if csv_value.size > 0:
            print(f'Average Precipitation Value: {csv_value[0]:.2f} mm')
        else:
            print("No data available in the CSV for this date.")

        # Get the corresponding GeoTIFF file
        date_key = selected_date.replace("-", "")  # Convert to YYYYMMDD for matching
        
        # Update the prefix based on the selected dataset
        if selected_dataset == "Agriculture":
            prefix = "ac_ay"
        elif selected_dataset == "Winter":
            prefix = "ac_wi"
        elif selected_dataset == "Growing Season":
            prefix = "ac_gs"
        else:
            prefix = ""

        selected_file = f"{prefix}_{date_key}.tif"  # Generate file name based on dataset
        
        # Check if the file exists in the directory
        if selected_file in os.listdir(data_paths[selected_dataset]):
            # Load the GeoTIFF data
            geo_file_path = os.path.join(data_paths[selected_dataset], selected_file)
            data = read_geotiff(geo_file_path)

            # Display the image
            plt.figure(figsize=(10, 5))
            plt.imshow(data, cmap='viridis')
            plt.colorbar(label='Precipitation (mm)')
            plt.title(f'Precipitation for {selected_dataset} on {selected_date} (GeoTIFF)')
            plt.xlabel('Pixel X')
            plt.ylabel('Pixel Y')
            plt.show()
        else:
            print("No GeoTIFF file available for this date.")

# Link the dataset dropdown to update the date options
dataset_dropdown.observe(update_dates, names='value')

# Link the date dropdown to the show_data function
date_dropdown.observe(show_data, names='value')

# Center the title using HTML
display(HTML("<h2 style='text-align: center;'>Select Precipitation Data</h2>"))

# Display centered widgets
display(widgets.VBox([dataset_dropdown, date_dropdown, output]))